# 신어후보 추출 Extractor.py 사용법

In [ ]:
import Extractor # Extractor 모듈 import
import pandas as pd
import sqlite3

In [ ]:
conn = sqlite3.connect('News.db') # 신어후보를 추출하고자 하는 db선택 News.db뿐만 아니라 Ent.db, HP.db가 있음
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head ORDER BY wdate DESC',conn) # db를 Dataframe으로 저장
conn.close()
df.drop_duplicates(keep='first', inplace=True) # 중복 게시글 제거


ext = Extractor.Ext(df) # Extractor 모듈 인스턴스 생성
df = ext.cleaning() # 불용어 처리

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True)) 
# extract_nouns에서 1차 신어후보를 찾고 search_dict에서 사전에 없는 단어를 추출하여 2차 신어후보 추출

sent = ext.extract_sent(new_words)
# 신어후보 단어가 들어가 있는 문장 추출

In [ ]:
# 감성분석에 필요한 변수 생성
statistic = ext.extract_statistic_value(sent)
# 위에서 신어후보가 들어가있는 문장을 추출하고 추출한 문장에서 감성분석에 필요한 8가지 변수 추출

r_rat = ext.extract_r_rat(sent,statistic)
# (신어후보의 오른쪽에 ~들이 붙은 비율 / 신어후보의 오른쪽에 그 외의 조사가 붙은 비율 / 신어후보의 오른쪽에 white space가 붙은 비율)
# 위 3가지 변수 추출

statistic = ext.combine_var(statistic, r_rat)
# statistic에서 추출한 8가지 변수와 r_rat에서 추출한 3가지 변수를 합쳐서 11개 변수 생성

conn = sqlite3.connect('News_var.db') 
statistic.to_sql('var', conn)
# 11개 변수를 저장할 db 생성하여 저장